# Why Generative Models?

This notebook is designed to provide motivation and intuition for why generative models exist, by showing how they relate to discriminative models (regressors, classifiers) that you may already be familiar with. We also explore why probabilistic models are a natural way to think about generation, we review standard methods to build generative models, and we end by talking about real-world applications of generative models.

In the way humans think, **discrimination** -- the ability to categorize and classify objects -- is usually very tightly linked to the ability to **generate** -- the ability to imagine or generate new kinds of objects. When you teach a child the digits 0 through 9, they are not only able to tell the different digits apart, but also conjure up images of the digits in their head when told to "think of the digit 7." 

However, on a purely algorithmic level, a machine that is able to discriminate successfully does not necessarily mean that it can also generate well. Consider a simple example of a machine that is designed to discriminate between the handwritten images of the digits $0$ and $1$. An algorithm may do quite well on this task simply by looking at whether the center pixel of the image is included in the handwritten stroke -- if it is, the algorithm predicts $1$; otherwise, it predicts $0$. Such a simple algorithm, as shown below, is quite accurate at discrimination: it gets XX accuracy discriminating between these two digits on the MNIST dataset. However, it hardly knows enough to generate new images of $0$ and $1$!

* Code

In some sense, discriminative models map samples to classes, whereas with a generative models, we'd like to go into the reverse direction: given a class, we'd like to be able to generate new images. In fact, this definition is only a special case of a generative model, known as the *conditional generative model*. More generally, or perhaps as a first step, it would nice to have a model that can produce a sample that looks like it could have been found in our original dataset, *from any class*.

## Probabilistic Methods 

In the way we have described generative models above, you'll notice something interesting: a generative model must be able to generate *more than one output*! That's kind of strange -- a discriminative model only needs to output a class for a given image, but for generative models to be interesting, a single output probably won't be very useful: it needs to generate many different samples of the digit $7$ for example. 

This is interesting, because that suggests we can't represent a generative model with a single deterministic function. If you think back to your middle-school algebra class, you'll remember a function can only have a single output. But we need some way to represent of our generative model to produce many different outputs. The standard way we do this -- though it is important to emphasize that this is a major modeling assumption -- is through *probabilistic models*. 

The idea is to define a random variable $X$ that takes values over the space of possible samples. We can say that this random variable has an associated random distribution $p_X(\cdot)$, which is zero for unrealistic samples, and non-zero for realistic or possible samples. We have essentially represented the 'output' of the generative model as an *outcome* of the random variable.

Probabilistic models have several good properties that make them good choices for generative models:

* **Sampling:** as described already, probabilistic models have
* **Learning:**
* **Conditioning:**
* **Inference:**
* **Uncertainty Modeling:**

## Standard Methods for Building Generative Models

I haven't talked about how we go from a dataset of samples to a probability distribution $p_X()$ yet. That will the topic of many of the later notebooks, but here is a quick summary of some standard methods:

* **Autoregressive Models**:
* **Variational Autoencoders**:
* **Flow Models**:
* **Generative Adversarial Networks:**

## Real-World Applications

"But how is this used in real life?" If you're wondering that, here are some ways generative models are used by researchers and in industry:

* **Speech synthesis**:
* **Interpolation**: This is a harder problem that it may look!

You'll see a lot more applications in the later notebooks!